content:      22:00 - 23:30
github order: 00:00 - 00:30
graphics:     00:30 - 01:00
GOOD NIGHT!

<p float="center">
 <img src="Resources/EverCompliantLOGO.png" width="300" hspace="50"/> 
 <img src="Resources/BarIlanLOGO.jpg" width="240" hspace="50"/> 
</p>
fix this!

# Classification Feedback Loop PoC - Project Paper

present: course, project, company, participants

company scenario and project goals. add project plan!

what is active learn. short review

EDA, include data presentation

data preprocess

In [1]:
import og